In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import avg, count, year

In [2]:
spark = SparkSession.builder \
    .appName("Steam Games Analysis") \
    .config("spark.executor.memory", "4g") \
    .config("spark.driver.memory", "4g") \
    .enableHiveSupport() \
    .getOrCreate()

24/09/20 15:35:39 WARN Utils: Your hostname, kitkat-virtual-machine resolves to a loopback address: 127.0.1.1; using 192.168.19.129 instead (on interface ens33)
24/09/20 15:35:39 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/09/20 15:35:42 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
spark.sparkContext.setLogLevel("ERROR")

In [4]:
def load_data(query):
    spark.sql("USE default")  
    return spark.sql(query).toPandas()

In [5]:
top_performing_games_query = """
SELECT Name, `Estimated owners`, `Peak CCU`
FROM steam_games 
ORDER BY `Estimated owners` DESC 
LIMIT 10
"""
top_performing_games_df = load_data(top_performing_games_query)
top_performing_games_df

,Name,Estimated owners,Peak CCU
0,Counter-Strike: Global Offensive,50000000 - 100000000,825215
1,Black Myth: Wukong,50000000 - 100000000,1214621
2,New World,50000000 - 100000000,16688
3,PUBG: BATTLEGROUNDS,50000000 - 100000000,275374
4,Team Fortress 2,50000000 - 100000000,107702
5,Call of Duty®: Modern Warfare® 2 (2009),5000000 - 10000000,545
6,NARAKA: BLADEPOINT,5000000 - 10000000,61338
7,Grim Dawn,5000000 - 10000000,4429
8,Braveland,5000000 - 10000000,2
9,Sea of Thieves,5000000 - 10000000,16818


In [7]:
pricing_trends_query = """
SELECT Price, Discount, COUNT(Name) AS game_count 
FROM steam_games 
GROUP BY Price, Discount 
ORDER BY game_count DESC
"""
pricing_trends_df = load_data(pricing_trends_query)
pricing_trends_df

,Price,Discount,game_count
0,0.00,0,19716
1,4.99,0,8314
2,9.99,0,7548
3,0.99,0,6818
4,1.99,0,5558
...,...,...,...
680,500.00,0,1
681,32.00,0,1
682,12.52,0,1
683,109.99,0,1


In [12]:
release_date_impact_query = """
SELECT year AS release_year, COUNT(Name) AS game_count, AVG(`User score`) AS avg_user_score 
FROM steam_games 
GROUP BY year 
ORDER BY year DESC
"""
release_date_impact_df = load_data(release_date_impact_query)
release_date_impact_df

,release_year,game_count,avg_user_score
0,2025.0,2,0.000000
1,2024.0,12580,0.000000
2,2023.0,15542,0.000000
3,2022.0,13989,0.000000
4,2021.0,12392,0.000000
5,2020.0,9644,0.007982
6,2019.0,7825,0.050339
7,2018.0,8183,0.247220
8,2017.0,6333,0.108164
9,2016.0,4415,0.034194


In [17]:
platform_distribution_query = """
SELECT  SUM(CASE WHEN Windows = 'True' THEN 1 ELSE 0 END) AS Windows_games,
        SUM(CASE WHEN Mac = 'True' THEN 1 ELSE 0 END) AS Mac_games,
        SUM(CASE WHEN Linux = 'True' THEN 1 ELSE 0 END) AS Linux_games
FROM steam_games
"""
platform_distribution_df = load_data(platform_distribution_query)
platform_distribution_df

,Windows_games,Mac_games,Linux_games
0,97377,17736,12332


In [20]:
user_engagement_query = """
SELECT Name, AVG(`Average playtime forever`) AS avg_playtime, AVG(`Median playtime forever`) AS median_playtime
FROM steam_games 
GROUP BY Name
ORDER BY avg_playtime DESC
LIMIT 10
"""
user_engagement_df = load_data(user_engagement_query)
user_engagement_df

,Name,avg_playtime,median_playtime
0,Boom 3D,145727.0,145727.0
1,Energy Engine PC Live Wallpaper,104238.0,208473.0
2,副作用之瞳-Tlicolity Eyes-,90351.0,90351.0
3,Defense Clicker,76068.0,76068.0
4,Relive,68357.0,136629.0
5,Sisyphus Reborn,68159.0,136291.0
6,YoloMouse,64973.0,114016.0
7,WARRIORS ALL-STARS,51388.0,102435.0
8,拯救大魔王2 Rescue the Great Demon 2,49555.0,99108.0
9,Combat Mission Shock Force 2,47336.0,47336.0


In [22]:
reviews_scores_query = """
SELECT Name, Reviews, `Metacritic score`, `User score` 
FROM steam_games 
ORDER BY `User score` DESC 
LIMIT 10
"""
reviews_scores_df = load_data(reviews_scores_query)
reviews_scores_df

,Name,Reviews,Metacritic score,User score
0,Hentai IQ Puzzle,None,0,100
1,K Station,“Kudos to the brilliant writing done in this g...,0,100
2,The Tower of Five Hearts,None,0,100
3,Hentai Strip Shot,None,0,100
4,Funbag Fantasy,None,0,100
5,DEEP SPACE WAIFU: NEKOMIMI,None,0,98
6,Meltys Quest,“A title possessing not only splendid scenes b...,0,97
7,Unlock Me,None,0,96
8,Ladykiller in a Bind,“This is the sex game we need in 2016.” Gamasu...,73,95
9,Deep Space Waifu: FLAT JUSTICE,None,0,95


In [29]:
Performance_by_Genre_query = """
SELECT Genres, COUNT(Name) AS game_count
FROM steam_games 
GROUP BY Genres
ORDER BY game_count DESC
LIMIT 10;
"""
Performance_by_Genre = load_data(Performance_by_Genre_query)
Performance_by_Genre

,Genres,game_count
0,"Casual,Indie",5484
1,None,4840
2,"Action,Indie",4755
3,"Action,Adventure,Indie",4082
4,"Adventure,Indie",3470
5,"Adventure,Casual,Indie",2891
6,Indie,2837
7,Casual,2796
8,"Action,Casual,Indie",2782
9,Action,2591


In [30]:
Genre_Comparison_User_Score_query = """
SELECT Genres, AVG(`User score`) AS avg_user_score, COUNT(Name) AS game_count 
FROM steam_games 
GROUP BY Genres
ORDER BY avg_user_score DESC
LIMIT 10;
 """
Genre_Comparison_User_Score = load_data(Genre_Comparison_User_Score_query)
Genre_Comparison_User_Score

,Genres,avg_user_score,game_count
0,"Sexual Content,Nudity,Indie",95.000000,1
1,"Casual,Free to Play,Indie,RPG,Simulation,Strategy",5.750000,8
2,"Action,Adventure,Free to Play,Indie,Simulation",4.368421,19
3,"Adventure,Casual,Free to Play,RPG",3.562500,16
4,"Action,Adventure,Indie,Massively Multiplayer",3.500000,22
5,"Adventure,Casual,Indie,RPG,Simulation,Strategy",1.058394,137
6,"Indie,RPG,Strategy",0.219697,396
7,"Casual,Strategy",0.210066,457
8,RPG,0.186874,899
9,"Casual,Simulation",0.157303,534


In [31]:
Genre_Based_Engagement_query = """
SELECT Genres, AVG(`Average playtime forever`) AS avg_playtime, AVG(`Median playtime forever`) AS median_playtime
FROM steam_games 
GROUP BY Genres
ORDER BY avg_playtime DESC
LIMIT 10;
"""
Genre_Based_Engagement = load_data(Genre_Based_Engagement_query)
Genre_Based_Engagement

,Genres,avg_playtime,median_playtime
0,"Utilities,Video Production,Web Publishing,Game...",36771.000000,65792.000000
1,"Animation & Modeling,Design & Illustration,Edu...",12994.600000,22803.200000
2,"Action,Free to Play,Indie,Massively Multiplaye...",9026.666667,5.000000
3,"Casual,Free to Play,Indie,Massively Multiplaye...",7840.000000,100.000000
4,"Audio Production,Utilities",7285.173913,6761.347826
5,"Education,Software Training,Utilities,Video Pr...",6774.333333,8727.333333
6,"Adventure,Casual,Free to Play,RPG",5656.312500,5654.625000
7,"Action,Casual,Simulation,Sports,Strategy",5356.750000,5356.750000
8,"Nudity,Design & Illustration",5095.000000,5095.000000
9,"Animation & Modeling,Design & Illustration,Uti...",4581.333333,5847.666667


In [32]:
Developers_Released_query = """
SELECT Developers, COUNT(Name) AS game_count 
FROM steam_games 
GROUP BY Developers
ORDER BY game_count DESC
LIMIT 10;
"""
Developers_Released = load_data(Developers_Released_query)
Developers_Released

,Developers,game_count
0,None,4872
1,Choice of Games,159
2,EroticGamesClub,157
3,Laush Dmitriy Sergeevich,149
4,Creobit,138
5,Boogygames Studios,123
6,Hosted Games,107
7,Sokpop Collective,103
8,"KOEI TECMO GAMES CO., LTD.",102
9,"Atomic Fabrik,Cristian Manolachi",99


In [33]:
Highest_Average_User_Score_query = """
SELECT Developers, AVG(`User score`) AS avg_user_score, COUNT(Name) AS game_count 
FROM steam_games 
GROUP BY Developers
ORDER BY avg_user_score DESC
LIMIT 10;
"""
Highest_Average_User_Score = load_data(Highest_Average_User_Score_query)
Highest_Average_User_Score

,Developers,avg_user_score,game_count
0,Maya Games,100.0,1
1,"NEKOPURU,ジーメン",92.0,1
2,R I MAD,88.0,1
3,"7DOTS,Rock Frog",82.0,1
4,Boobs Dev,77.0,1
5,"Pirotexnik,TeeRax",69.0,1
6,"HawkX Games,Double Mirage Studio",57.0,1
7,1bit,55.0,1
8,Toothless T-Rex,51.0,1
9,TsukiWare,50.0,2
